In [185]:
import torch
import torch.nn as nn
import math

In [10]:
encoder_layer = nn.TransformerEncoderLayer(d_model=512, nhead=8)

In [11]:
transformer_encoder = nn.TransformerEncoder(encoder_layer, num_layers=6)

In [132]:
src = torch.rand(32, 40, 768)

In [324]:
class MultiHeadSelfAttn(nn.Module):
    def __init__(self, in_features, out_features, heads=8):
        '''
        in_features: should be equal to the embedding_dimension
        output: what the size of the output embedding should be
        '''
        
        super(MultiHeadSelfAttn, self).__init__()
        self.in_features = in_features
        self.heads = heads
        self.out_features = out_features
        
        # Make sure that in_features is compatible with the number of heads
        assert in_features % heads == 0
        
        # dk is the size of each of the linear projections of the embedding
        self.dk = in_features // 8
        
        # These are the parameters to project the matrix to the amount of heads
        self.key_projections = nn.Linear(self.in_features, self.in_features)
        self.value_projections = nn.Linear(self.in_features, self.in_features)
        self.query_projections = nn.Linear(self.in_features, self.in_features)
        
        # The final linear layer
        self.end_linear = nn.Linear(self.in_features, self.out_features)
        
        # Softmax
        self.softmax = nn.Softmax(dim=1)
        
    def scaled_attention(self, head_query, head_keys, head_values):
        # Calculate the scaled dot-product
        attn = torch.matmul(head_query, head_keys.transpose(-2, -1)) / math.sqrt(self.dk)
        
        # Get the softmax
        attn = self.softmax(attn)
        
        # Multiply the softmax output by the values
        attn = torch.matmul(attn, head_values)
        return attn
        
    def forward(self, embeddings):
        '''
        Forward propagate through the multi-head attention
        embeddings: should be of dimensions (batch, sequence_length, embedding_dimension)
        '''
        
        batches, sequence_length, embeddings_dim = embeddings.size()
        
        # Get the query projections
        query = self.query_projections(embeddings)
        query = query.view(batches, self.heads, sequence_length, self.dk)
        
        # Get the key projections
        keys = self.key_projections(embeddings)
        keys = keys.view(batches, self.heads, sequence_length, self.dk)
        
        # Get the value projections
        values = self.value_projections(embeddings)
        values = values.view(batches, self.heads, sequence_length, self.dk)
        
        # Calculated the scaled dot-product attention
        attn_out = self.scaled_attention(query, keys, values)
        
        # Put it in dimensions (batches, sequence_length, in_features)
        attn_out = attn_out.view(batches, sequence_length, self.in_features)
        
        # Apply the final linear layer
        return self.end_linear(attn_out)
        

In [325]:
multi_attn = MultiHeadSelfAttn(768, 768)
x = torch.randn(32, 40, 768)
multi_attn(x).size()

torch.Size([32, 40, 768])

In [336]:
class PositionWiseFeedForward(nn.Module):
    def __init__(self, in_features, inner_features, out_features, dropout=0.1):
        super(PositionWiseFeedForward, self).__init__()
        self.in_features = in_features
        self.inner_features = inner_features
        self.out_features = out_features
        
        # First linear layer goes from the in_features to inner_features dimension
        self.fc1 = nn.Linear(in_features, inner_features)
        
        # Second linear layer goes from inner_features to out_features
        self.fc2 = nn.Linear(inner_features, out_features)
        self.relu = nn.ReLU()
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, x):
        '''
        Forward propagate through the Feed-Forward network
        '''
        
        x = self.fc1(x)
        x = self.dropout(x)
        x = self.fc2(x)
        x = self.dropout(x)
        return x

In [337]:
feed_forward = PositionWiseFeedForward(768, 2560, 768)
x = torch.randn(32, 40, 768)
feed_forward(x).size()

torch.Size([32, 40, 768])